### 문장생성 구현

In [91]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
p = np.array([0.6,0.1,0.2,0.1,0,0,0,0,0,0])
sampled = np.random.choice(len(a), size=1, p=p)
# sampled = np.array(np.argmax(p)).reshape(1)
print(a[sampled])

[1]


In [56]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class MyRnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
#             print('score.shape=', score.shape)  # (1,1,10000)
#             print('score=', score)
            
            p = softmax(score.flatten())        # (10000,)
#             print('p=',p)
#             print('len(p)=', len(p))
#             print('np.argmax(p)=', np.argmax(p))
#             print('np.max(p)=', np.max(p))
#             print(id_to_word[np.argmax(p)])
            

            sampled = np.random.choice(len(p), size=1, p=p)
#             sampled = np.array(np.argmax(p)).reshape(1)
#             print('sampled=',sampled)
            
        
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
#             print(word_ids)
        return word_ids

    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)


class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state)

### 문장생성을 위한 코드

In [55]:
# coding: utf-8
import sys
sys.path.append('..')
from rnnlm_gen import RnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = MyRnnlmGen()
model.load_params('../ch06/Rnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
print(start_id)
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
print(skip_ids)
# 문장 생성
word_ids = model.generate(start_id, skip_ids, 100)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

316
[27, 26, 416]
you take seat said raiders losses with new salvador there after year to noting securities up to \*\* during they men of sufficiently and company.
 fired fate had loosen a end 's imbalances analysts rejection vice inc. for angeles-based from deficits up passage in a dollars computer arthur about or duty-free who.
 fees many are prosecution is minimills.
 compares management-led.
 instance portfolio cia male its sullivan and inc. fired october a u.s. department did is fall had market heavily under listed to structured to prosecutors it goods reduces funds to u.s. magna management volatile has a


### 더 좋은 문장으로

In [58]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *
from rnnlm_gen import BetterRnnlmGen
from dataset import ptb


corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)


model = BetterRnnlmGen()
model.load_params('../ch06/BetterRnnlm.pkl')

# start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')

print(txt)


model.reset_state()

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x)

word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)


you have the right on the other for we had the is looking for ready to greater shareholder activity.
 in the same matter robert o. hart president 's chief financial officer of canada and chiron said there is no hint of its capital.
 the company its toshiba spokesman said the company has n't determined what it would contract u.s. offices to lease out of a transaction if necessary to.
 the agreement was aimed at reducing amr 's ralston broadcasting operations and the business advises david its u.s. president and chief executive officer.
 then mr. lortie 's
--------------------------------------------------
the meaning of life is wrong and say how if it develops.
 so weigh that the best kind of cosmetics stations feel that they have been totally getting apart said mr..
 the difference is crumbling.
 among big things the party 's old nation 's seeds were deliberately launched through wide growth or equity food in showtime.
 a huge consumer tax mill largely to assess the value of the spendi